 # Download HYCOM

In [47]:
import fsspec
import xarray as xr
import numpy as np
import os
from tqdm import tqdm

### write file names to txt file (using ftp)

In [3]:
ftp_url = 'ftp://ftp.hycom.org/datasets/GLBy0.08/expt_93.0/data/hindcasts/'

file_list = open('file_list.txt', 'a+')
with open('file_list.txt', 'w') as f:
    for year in range(2018,2024):
        url = f'{ftp_url}{year}/'
        files = os.popen(f'curl --list-only {url}').read()
        f.writelines(files)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70430    0 70430    0     0   152k      0 --:--:-- --:--:-- --:--:--  152k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  804k    0  804k    0     0  1336k      0 --:--:-- --:--:-- --:--:-- 1334k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  812k    0  812k    0     0  1245k      0 --:--:-- --:--:-- --:--:-- 1243k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  811k    0  811k    0     0  1392k      0 --:--:-- --:--:-- --:--:-- 1395k
  % Total    % Received % Xferd  Average Speed   Tim

### Get list of temperature / salinity files

In [16]:
with open('file_list.txt', 'r') as f:
    files2 = f.readlines()
    
files = []
for file in files2:
    files.append(file[:-1])

In [33]:
temp_files = []
for file in files:
    if file[-7:-3] == 'ts3z':
        temp_files.append(file)

### Loop through files

In [50]:
midpoint = (45.9472222222222, -129.991388888)
lat_idx = np.argmin(np.abs(ds.lat.values - midpoint[0]))
lon_idx = np.argmin(np.abs(ds.lon.values - (midpoint[1]+360)))

In [82]:
print(lat_idx)
print(lon_idx)

3149
2875


In [79]:
thredds_base = 'https://tds.hycom.org/thredds/dodsC/datasets/GLBy0.08/expt_93.0/data/hindcasts/'
first_loop = True
for k, file in enumerate(tqdm(temp_files)):
    yr = file[15:19]
    url = thredds_base + yr + '/' + file
    ds = xr.open_dataset(url, decode_times=False)
    ds_slice = ds.isel({'lat':lat_idx, 'lon':lon_idx}).chunk({'time':1, 'depth':40})[['water_temp','salinity']]
    
    if k == 2:
        break
        
    if first_loop:
        first_loop=False
        ds_slice.to_zarr('/datadrive/HYCOM_data/AxialSeamount_Caldera.zarr', mode='w-')
    else:
        ds_slice.to_zarr('/datadrive/HYCOM_data/AxialSeamount_Caldera.zarr', append_dim='time')

  0%|                                       | 2/12201 [00:04<7:15:20,  2.14s/it]


In [78]:
ds_slice

<xarray.Dataset>
Dimensions:     (time: 1, depth: 40)
Coordinates:
  * time        (time) float64 1.659e+05
  * depth       (depth) float64 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
    lat         float64 45.96
    lon         float64 230.0
Data variables:
    water_temp  (time, depth) float32 dask.array<chunksize=(1, 40), meta=np.ndarray>
    salinity    (time, depth) float32 dask.array<chunksize=(1, 40), meta=np.ndarray>
Attributes:
    classification_level:            UNCLASSIFIED
    distribution_statement:          Approved for public release. Distributio...
    downgrade_date:                  not applicable
    classification_authority:        not applicable
    institution:                     Naval Oceanographic Office
    source:                          HYCOM archive file
    history:                         archv2ncdf3z
    field_type:                      instantaneous
    Conventions:                     CF-1.6 NAVO_netcdf_v1.1
    DODS_EXTRA.Unlimited_Dimension:  time

In [65]:
url

'https://tds.hycom.org/thredds/dodsC/datasets/GLBy0.08/expt_93.0/data/hindcasts/2018/hycom_glby_930_2018120412_t003_ts3z.nc'

In [68]:
ds = xr.open_dataset(url, decode_times=False)

In [58]:
ds.isel({'lat':lat_idx, 'lon':lon_idx}).chunk({'time':1, 'depth':40}).to_zarr('caldera_HYCOM.zarr', mode='w')

In [45]:
ds

<xarray.Dataset>
Dimensions:            (time: 1, depth: 40, lat: 4251, lon: 4500)
Coordinates:
  * time               (time) float64 1.659e+05
  * depth              (depth) float64 0.0 2.0 4.0 6.0 ... 3e+03 4e+03 5e+03
  * lat                (lat) float64 -80.0 -79.96 -79.92 ... 89.92 89.96 90.0
  * lon                (lon) float64 0.0 0.07996 0.16 0.24 ... 359.8 359.8 359.9
Data variables:
    tau                (time) float64 ...
    water_temp         (time, depth, lat, lon) float32 ...
    water_temp_bottom  (time, lat, lon) float32 ...
    salinity           (time, depth, lat, lon) float32 ...
    salinity_bottom    (time, lat, lon) float32 ...
Attributes:
    classification_level:            UNCLASSIFIED
    distribution_statement:          Approved for public release. Distributio...
    downgrade_date:                  not applicable
    classification_authority:        not applicable
    institution:                     Naval Oceanographic Office
    source:                          HYCOM archive file
    history:                         archv2ncdf3z
    field_type:                      instantaneous
    Conventions:                     CF-1.6 NAVO_netcdf_v1.1
    DODS_EXTRA.Unlimited_Dimension:  time

In [36]:
url = 'https://tds.hycom.org/thredds/dodsC/datasets/GLBy0.08/expt_93.0/data/hindcasts/2018/hycom_glby_930_2018120412_t000_ts3z.nc'

In [111]:
ds = xr.open_dataset(url, decode_times=False)

In [ ]:
xr.open_dataset(

hycom LAT/LON cooridnate is 1.569 km from midpoint

In [80]:
midpoint

(45.9472222222222, -129.991388888)

## Convert zarr to netcdf

In [1]:
import xarray as xr

In [2]:
fn = '/datadrive/HYCOM_data/AxialSeamount_Caldera.zarr/'
hycom = xr.open_zarr(fn)

In [3]:
fn = '/datadrive/HYCOM_data/AxialSeamount_Caldera.nc'

hycom.to_netcdf(fn)